# Importar librerías

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from fbprophet import Prophet

# Leer datos

Trabajaremos con el conjunto de datos Box and Jenkins (1976) Airline Passengers (Pasajeros aéreos), que contiene datos de serie de tiempo del número mensual de pasajeros entre 1949 y 1960.

In [ ]:
df = pd.read_csv('AirPassengers.csv')

df.head(5)

# Adecuar los datos al formato requerido por Prophet

Prophet requiere que la columna de tiempo esté en el formato "datetime""

In [ ]:
df.dtypes

In [ ]:
df['Month'] = pd.DatetimeIndex(df['Month'])
df.dtypes

También requiere que las columnas se llamen "ds" y "y", respectivamente

In [ ]:
df = df.rename(columns={'Month': 'ds',
                        'AirPassengers': 'y'})

df.head(5)

# Visualización de los datos

Siempre es buena práctica ver cómo son los datos

In [ ]:
ax = df.set_index('ds').plot(figsize=(12, 8))
ax.set_ylabel('Número de pasajeros aéreos por mes')
ax.set_xlabel('Fecha')

plt.show();

# Predicción con Prophet

Debemos instanciar un objeto de la Clase Prophet

In [ ]:
# fijar el intervalo de incertidumbre en 95% (el default de Prophet es 80%)
mi_modelo = Prophet(interval_width = 0.95)

Ajustamos el modelo con los datos del dataframe

In [ ]:
mi_modelo.fit(df)

Para obtener predicciones, se debe provee al Prophet un nuevo DataFrame que contenga una columna "ds" con las fechas para las cuales se quiere una predicción.  No es necesario crear este nuevo DataFrame manualmente ya que Prophet contiene una función que lo hace:  make_future_dataframe()

Le indicaremos que queremos 36 nuevas fechas que representan meses

In [ ]:
fechas_futuras = mi_modelo.make_future_dataframe(periods=36, freq='MS')
fechas_futuras.tail()

Este DataFrame de fechas futuras se utiliza como entrada al método "predict" de nuestro modelo ajustado.

In [ ]:
prediccion = mi_modelo.predict(fechas_futuras)

El Prophet devuelve un DataFrame grande con muchas columnas interesantes.  Sin embargo, vamos a pedirle solo aquellas columnas que son las más relevantes para la predicción.  Estas son:

ds: el tiempo de el valor predicho
yhat: tel valor predicho de nuestra métrica (en la Estadística, yhat es una notación que tradicionalmente se usa para representar los valores predichos de un valor y )
yhat_lower: el límite inferior de nuestras predicciones
yhat_upper: el límite superior de nuestras predicciones

In [ ]:
prediccion[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

Es de esperar variaciones en los resultados que se ven arriba ya que Prophet se vale de métodos 
Markov Chain Monte Carlo (MCMC) para generar las predicciones.  MCMC es un proceso estocástico, por lo que los valores serán levemente diferentes cada vez que se ejecuta.

Prophet también provee una conveniente función para graficar los resultados de la predicción

In [ ]:
mi_modelo.plot(prediccion,
               uncertainty=True);

La gráfica muestra los valores observados de nuestra serie de tiempo (los puntos negros), los valored predichos (la línea azul) y los intervalos de incertiddumbres de las predicciones (las regiones sombreadas en azul).

Una característica particularmente fuerte de Prophet es su habilidad para mostrar los componentes de las predicciones.  Esto puede ayudar a revelar cómo contribuyen los patrones diarios, semanales y anuales de la serie de tiempo al valor global, o total, de las predicciones: 

In [ ]:
mi_modelo.plot_components(prediccion);